In [1]:
import numpy as np
import numpy as np
import pandas as pd
#import WindFarmGenetic_v1 # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver1_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
n_mc_samples=10000 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 200  # number of cells each row
rows_cells = 200  # number of cells each column
cell_width = 20 # unit (TBD)--> have to be computed.
constraint_dist = 400 #unit distance between two turbines
constraint_perim = 50

radius=np.ceil(constraint_dist/cell_width)

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=2 #number of independent runs with random initialization.


In [5]:
perim_cells=int(np.ceil(constraint_perim/cell_width)) #TBU generalize it for multiple boundary cells.
NA_loc=[] 

for curr_perim_cell in range(perim_cells):
    NA_loc.extend([i+cols_cells*curr_perim_cell for i in range(cols_cells)]) #upper row
    NA_loc.extend([i*(cols_cells)+curr_perim_cell for i in range(rows_cells)]) #left boundary
    NA_loc.extend([i*(cols_cells)+cols_cells-1-curr_perim_cell for i in range(rows_cells)])#right Boundary
    NA_loc.extend([(cols_cells)*(rows_cells-1-curr_perim_cell)+i for i in range(rows_cells)])#down Boundary


In [8]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
svr_model_filename = 'svr_1s1d_N_13.svr' #svr training file.
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)

In [9]:
import WindFarmGenetic_v1 as WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate,constraint_dist=constraint_dist)

In [10]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

Generation layout: 100%|██████████| 100/100 [04:59<00:00,  2.99s/it]


In [8]:
##Generate initial layouts: single layout--> 1d array of size row*col, layouts are set of num_training_sample_SVM different(random) layouts.  
train_mc_layouts, train_mc_layouts_NA = WindFarmGenetic.LayoutGridMCGenerator.gen_mc_grid_with_NA_loc_circle(rows_cells,
                                                                                                      cols_cells,
                                                                                                      n_mc_samples,
                                                                                                      wt_N, NA_loc,radius,
                                                                                                      "{}/mc_layout.dat".format(wds_data_folder),
                                                                                                      "{}/mc_layout_NA.dat".format(wds_data_folder))

layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter="  ", dtype=np.int32)

Generation layout: 100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


In [9]:

wfg.mc_gen_xy_NA_hackathon(rows=rows_cells, cols=cols_cells, layouts=layouts, n=n_mc_samples, N=wt_N,  xfname="{}/x.dat".format(wds_data_folder),
                 yfname="{}/y.dat".format(wds_data_folder))

layouts: 100%|██████████| 100/100 [00:23<00:00,  4.30it/s]


In [9]:
x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
x_original = x_original.values

In [10]:
y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
y_original = y_original.values.flatten()

In [18]:
y_original

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [11]:
print(x_original.shape)
print(y_original.shape)

(6400, 2)
(6400,)


In [12]:
# create a SVR object and specify the kernal and other parameters
svr_model = SVR(kernel='rbf', C=1.0, gamma=0.3, epsilon=.1)
# build the SVR power distribution model
svr_model.fit(x_original, y_original)

# save the SVR model to a file
pickle.dump(svr_model, open("{}/{}".format(wds_data_folder, svr_model_filename), 'wb'))

In [17]:
len(svr_model.support_vectors_)

5709

In [ ]:
del x_original
del y_original

In [11]:
svr_model=pickle.load(open("{}/{}".format(wds_data_folder, svr_model_filename), 'rb'))

In [12]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, svr_model=svr_model, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...


layouts:   0%|          | 0/100 [00:00<?, ?it/s]

Support vector regression guided genetic algorithm starts....
generation 0...


layouts: 100%|██████████| 100/100 [00:36<00:00,  2.75it/s]


[512.5478  511.74945 512.4323  515.668   511.52927 510.88672 510.31277
 511.12906 513.112   511.2019  512.4457  510.79144 512.5853  511.29016
 510.34995 509.8945  509.66077 509.8265  510.85626 512.92816 513.2006
 509.76382 512.6866  513.6231  511.95502 510.7786  511.08954 509.1975
 510.48334 514.6012  507.2884  511.16318 511.6991  510.5968  511.3447
 509.70233 516.3011  511.01605 513.7068  509.25604 510.85306 511.38504
 510.102   513.55084 512.4496  514.30035 511.64764 509.7853  510.9809
 506.83414 515.0243  512.8646  511.25433 508.54865 511.93045 510.86374
 514.50476 511.387   513.1675  509.2618  512.4461  513.2064  510.9592
 508.16098 510.73007 511.00497 512.8911  514.20245 512.3146  511.76807
 513.7142  512.6128  511.93118 508.14484 513.9519  508.12094 512.6638
 512.57477 512.6914  512.31445 509.956   510.52158 515.4132  512.4242
 512.96844 512.60925 509.98294 511.6789  510.78842 513.38226 508.28293
 510.48782 514.51733 512.4829  510.63705 514.67914 510.68262 511.6019
 511.99838 511

...

layouts:   0%|          | 0/100 [00:00<?, ?it/s]

generation 10...


layouts: 100%|██████████| 100/100 [00:38<00:00,  2.59it/s]


[519.45575 517.7118  515.7901  517.3835  516.28186 517.30194 515.9657
 513.6149  517.61993 517.2393  517.3978  517.7752  517.72894 516.8426
 514.67615 517.2003  516.5599  516.37537 518.9273  516.05756 517.0023
 517.8256  515.92413 516.538   517.2612  517.837   517.543   516.6177
 517.1124  517.51575 515.70807 516.3129  516.8665  517.422   517.1398
 517.9289  518.30334 517.18274 515.58185 516.4667  516.2162  516.56146
 516.9147  517.8836  515.6522  517.493   517.5024  517.8674  517.2822
 517.7607  517.2778  516.98706 517.12305 517.2811  516.16364 516.9879
 516.5503  517.9322  512.6656  516.832   516.4563  516.79926 518.61115
 515.55066 515.359   517.71277 516.2917  516.6496  517.2219  517.1982
 517.47955 516.9317  517.949   518.1195  517.7796  516.6355  516.6294
 517.6909  515.5949  519.0239  517.3667  516.1634  515.6134  516.7571
 516.7354  516.88684 515.83435 515.5423  515.0832  517.6386  514.5619
 514.7555  516.7513  517.101   517.3427  517.2377  516.1378  516.57184
 516.748   518.74

KeyboardInterrupt: 

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797